<a href="https://colab.research.google.com/github/Martim-21805485/IASolucoes/blob/main/Lab10_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 10
Stochastic Search

## Purpose
Study of a informed search algorithm

## Methodology
Implementation

## Results
An implementation of Simulated Annealing in Python 

# Setup

## Library import
We import all the required Python libraries

In [ ]:
import random
import math

import pandas as pd

# Parameters

In [ ]:
TSP = {
    "Lisboa": {
        "Aveiro": 255,
        "Beja": 178,
        "Castelo Branco": 224,
        "Vilamoura": 263,
        "Porto": 356
    },
    "Aveiro": {
        "Lisboa": 255,
        "Beja": 389,
        "Castelo Branco": 200,
        "Vilamoura": 475,
        "Porto": 80
    },
    "Beja": {
        "Aveiro": 389,
        "Lisboa": 178,
        "Castelo Branco": 275,
        "Vilamoura": 130,
        "Porto": 451
    },
    "Castelo Branco": {
        "Aveiro": 200,
        "Beja": 275,
        "Lisboa": 224,
        "Vilamoura": 310,
        "Porto": 170
    },
    "Vilamoura": {
        "Aveiro": 475,
        "Beja": 130,
        "Castelo Branco": 310,
        "Lisboa": 263,
        "Porto": 570
    },
    "Porto": {
        "Aveiro": 80,
        "Beja": 451,
        "Castelo Branco": 170,
        "Lisboa": 356,
        "Vilamoura": 570
    },
}

In [ ]:
pd.DataFrame(TSP).T.fillna(0)

,Aveiro,Beja,Castelo Branco,Vilamoura,Porto,Lisboa
Lisboa,255.0,178.0,224.0,263.0,356.0,0.0
Aveiro,0.0,389.0,200.0,475.0,80.0,255.0
Beja,389.0,0.0,275.0,130.0,451.0,178.0
Castelo Branco,200.0,275.0,0.0,310.0,170.0,224.0
Vilamoura,475.0,130.0,310.0,0.0,570.0,263.0
Porto,80.0,451.0,170.0,570.0,0.0,356.0


## Pergunta 1

Escreva uma função `random_tour(graph, start)` a qual passamos o grafo TSP e o ponto de partida (chegada). 
A função deverá devolver um tuplo cuja primeira componente deve ser a sequencia de cidades
começado e acabando no mesmo lugar. O segundo elemento deve ser o custo desse tour. Para esta 
segunda componente deverá escrever uma segunda função `tour_cost(graph, tour)` que recebe o grafo TSP e 
o tour.  

### Dicas

1. Dado um dicionário `d` a função `d.keys()` é útil para obter todas as chaves definidas em `d`
2. A biblioteca `random` contem uma função `shuffle` que poderá ser útil

In [ ]:
def tour_cost(graph, tour):
    """
    Esta função calcula o 'custo' total de um tour. No exemplo desta prática trabalhamos com distância.
    g: grafo
    tour: qualquer tour valido de acordo com o grafo g
    """

    # we can make use of the zip function
    cost = 0
    for city_start, city_end in zip(tour[:-1], tour[1:]):
        cost += graph[city_start][city_end]

    # or with a list comprehension
    cost = (
        sum([graph[city_start][city_end]
             for city_start, city_end in zip(tour[:-1], tour[1:])])
    )
    return cost


def random_tour(graph: dict, start: str) -> tuple:

    other_cities = [city for city in graph.keys() if city != start]
    random.shuffle(other_cities)
    tour = [start] + other_cities + [start]
    cost = tour_cost(graph, tour)

    return (tour, cost)

In [ ]:
random_tour(TSP, 'Lisboa')

(['Lisboa',
  'Porto',
  'Beja',
  'Vilamoura',
  'Castelo Branco',
  'Aveiro',
  'Lisboa'],
 1702)

## Pergunta 2

Considere a função `swap_random(lst)` dada para a sua conveniência.
Escreva uma função `neighbour_tour(g, tr, hist, max_it` que

1. Calcule um tour vizinho do tour `tr` que resulta de trocar duas cidades no itinerário. 
2. O tour vizinho não pode alterar o ponto de partida (chegada)
3. O tour vizinho não pode ser conhecido, por isso passamos uma list hist que contem os itinerários já conhecidos
4. Se a função não encontra um novo vizinho nunca antes visto em `max_it` iterações, a mesma devolverá o valor `-1`

In [ ]:
def swap_random(lst: list):
    """
    A simple implementation of a function that swaps a pair of random elements in a list
    
    lst: input list
    """
    idx = range(len(lst))
    i1, i2 = random.sample(idx, 2) 
    lst[i1], lst[i2] = lst[i2], lst[i1] 
    
    return lst


def neighbour_tour(graph: dict, tour: list, hist: list, max_it: int):
    """
    generate a neighbour tour by swapping a random pair of inner cities.
    the generated neighbour must be a tour not previously considered, thus not in the hist
    max_it is used to stop trying to find an unseen neighbour after max_it trials
    
    g: graph
    tour: current tour
    hist: previously seen tours
    max_it: max number of iterations after which we stop searching for unseen neighbours
    """
    # defina o corpo da sua função aqui
    # devolver tuplo correspondente a novo tour e custo se encontrar
    # se depois de max_it tentativas não encontrar novo tour
    # devolver o valor -1
    
    for _ in range(max_it):
        
        nb_tour = [tour[0]] + swap_random(tour[1:-1]) + [tour[-1]]
        
        # compute cost
        nb_cost = tour_cost(graph, nb_tour)  
        
        if (nb_tour, nb_cost) not in hist:
            
            # append to history
            hist.append( (nb_tour, nb_cost) )
            
            # return path, cost, and history
            return nb_tour, nb_cost, hist

    # If no path was found return None values for both tour and cost and the hist    
    return None, None, hist
       

Get a tour first

In [ ]:
tour, cost = random_tour(TSP, 'Beja')

Starts in 

In [ ]:
[tour[0]]

['Beja']

In [ ]:
tour, cost

(['Beja', 'Porto', 'Castelo Branco', 'Lisboa', 'Vilamoura', 'Aveiro', 'Beja'],
 1972)

Create a data structure to keep all the paths

In [ ]:
hist = []
hist.append(tour)

# Compute one path
a_tour, cost, hist = neighbour_tour(TSP, tour, hist, 30)
a_tour, cost, hist

(['Beja', 'Porto', 'Castelo Branco', 'Aveiro', 'Vilamoura', 'Lisboa', 'Beja'],
 1737,
 [['Beja', 'Porto', 'Castelo Branco', 'Lisboa', 'Vilamoura', 'Aveiro', 'Beja'],
  (['Beja',
    'Porto',
    'Castelo Branco',
    'Aveiro',
    'Vilamoura',
    'Lisboa',
    'Beja'],
   1737)])

Or create an empty data structure and capture all the possible paths

In [ ]:
hist = []
for _ in range(30):
    nb_tour, nb_cost, hist = neighbour_tour(TSP, tour, hist, 30)
    
    # if no more paths, stop
    if nb_tour is None:
        break
hist

[(['Beja', 'Porto', 'Vilamoura', 'Lisboa', 'Castelo Branco', 'Aveiro', 'Beja'],
  2097),
 (['Beja', 'Porto', 'Aveiro', 'Lisboa', 'Vilamoura', 'Castelo Branco', 'Beja'],
  1634),
 (['Beja', 'Lisboa', 'Castelo Branco', 'Porto', 'Vilamoura', 'Aveiro', 'Beja'],
  2006),
 (['Beja', 'Porto', 'Castelo Branco', 'Aveiro', 'Vilamoura', 'Lisboa', 'Beja'],
  1737),
 (['Beja', 'Porto', 'Castelo Branco', 'Vilamoura', 'Lisboa', 'Aveiro', 'Beja'],
  1838),
 (['Beja', 'Castelo Branco', 'Porto', 'Lisboa', 'Vilamoura', 'Aveiro', 'Beja'],
  1928),
 (['Beja', 'Porto', 'Lisboa', 'Castelo Branco', 'Vilamoura', 'Aveiro', 'Beja'],
  2205),
 (['Beja', 'Vilamoura', 'Castelo Branco', 'Lisboa', 'Porto', 'Aveiro', 'Beja'],
  1489),
 (['Beja', 'Porto', 'Castelo Branco', 'Lisboa', 'Aveiro', 'Vilamoura', 'Beja'],
  1705),
 (['Beja', 'Aveiro', 'Castelo Branco', 'Lisboa', 'Vilamoura', 'Porto', 'Beja'],
  2097)]

## Pergunta 3

Implemente o algoritmo de simulated annealing cujo pseudocódigo poderá encontrar no conjunto de slides de busca estocástica.


In [ ]:
def simulated_annealing(graph, start, t, alpha, max_it):
    """
    Perform simulated annealing stochastic search
    
    g: graph
    start: the city in which we start and end the tour
    t: temperature
    alpha: factor by which to decrease the temperature
    max_it: max it number of max iterations to search for unseen neighbours
    """
    
    #  g = the input graph
    #  history = []    
    g = graph
    history = []
    
    # h = tuple with some random tour that begins and ends in city s and total cost
    h = random_tour(graph, start)
    # append h to the history
    history.append(h)
    # set initial temperature to t
    temp_initial = t
    # set final temperature to a low value
    temp_final = 0.000001
    # current temp = initial temperature
    temp_current = temp_initial
    #  while current temp >= final temperature:
    while temp_current >= temp_final:
        
        # get current tour and cost
        tour, cost = h
        tour = h[0]
        cost = h[1]
        
        print("current temp: {}, cost: {}".format(temp_current, cost))
        
        # get a new neighbour tour that is not in history (and its cost)"
        nb_tour, nb_cost, history = neighbour_tour(graph, tour, history, max_it)
        
        # if new neighbour was found:"
        if nb_tour is not None:
            
            # delta = cost of current tour - cost of new tour
            delta = cost - nb_cost
            
            print("delta:{}".format(delta))
            # if delta is positive (means new tour is shorter):
            if delta > 0:
                print('new path found, greedy search')
                # current tour = new tour
                h = (nb_tour, nb_cost)
  
            # else, we activate the temperature mechanism
            else:
                # get a random real d between 0 and 1
                random_number = random.random()
                
                # if e < e^(delta / current temp) (e -> Euler’s number 2.7182..)
                print("delta:{}, random_number:{}, temp_current:{}, prob:{}".format(delta, random_number, temp_current, math.exp(delta / temp_current)))
                if random_number < math.exp(delta / temp_current):
                        print('new path found via TEMP')
                        # h = n
                        h = (nb_tour, nb_cost)
                    
        # decrement the temperature
        temp_current -= alpha
        
    return h


In [ ]:
simulated_annealing(TSP, "Beja", t=200, alpha=1, max_it=5)

current temp: 200, cost: 1382
delta:128
new path found, greedy search
current temp: 199, cost: 1254
delta:-420
delta:-420, random_number:0.49599698500830347, temp_current:199, prob:0.12117096899062368
current temp: 198, cost: 1254
delta:-255
delta:-255, random_number:0.41828244108124313, temp_current:198, prob:0.27585531054564477
current temp: 197, cost: 1254
delta:131
new path found, greedy search
current temp: 196, cost: 1123
delta:-545
delta:-545, random_number:0.4546987104458896, temp_current:196, prob:0.06200053624274233
current temp: 195, cost: 1123
delta:-380
delta:-380, random_number:0.7862925858112442, temp_current:195, prob:0.142456591217157
current temp: 194, cost: 1123
delta:-716
delta:-716, random_number:0.6452139068425198, temp_current:194, prob:0.024953987510745
current temp: 193, cost: 1123
delta:-327
delta:-327, random_number:0.4364002482928566, temp_current:193, prob:0.18372769828462274
current temp: 192, cost: 1123
delta:-134
delta:-134, random_number:0.8813857777508

(['Beja', 'Vilamoura', 'Castelo Branco', 'Lisboa', 'Porto', 'Aveiro', 'Beja'],
 1489)

In [ ]:
simulated_annealing(TSP, "Beja", t=0.001, alpha=0.001, max_it=5)

current temp: 0.001, cost: 2098
delta:134
new path found, greedy search


(['Beja', 'Porto', 'Vilamoura', 'Castelo Branco', 'Aveiro', 'Lisboa', 'Beja'],
 1964)

## Pergunta 4
### Se o tempo permitir, caso contrário fica como trabalho para casa:

No penúltimo slide do conjunto de slides de busca estocática temos uma versão estendida das cidades portuguesas, a qual inclui uma matriz triangular das distâncias entre onze cidades. Escreva uma função em Python que (a) leia uma matriz deste género em formato CSV (pode assumir que os valores não representados na matriz do slide são -1 e (b) gere uma lista de adjacencias como aquela que utilizou para esta sessão prática

In [ ]:
import numpy as np
import pandas as pd

example = pd.DataFrame(np.tril(np.random.randint(1,100,100).reshape(10,10), k=-1), 
                       columns=["City"+str(x) for x in range(10)])
example.to_csv('example.csv', index=False)

In [ ]:
example

,City0,City1,City2,City3,City4,City5,City6,City7,City8,City9
0,0,0,0,0,0,0,0,0,0,0
1,51,0,0,0,0,0,0,0,0,0
2,22,8,0,0,0,0,0,0,0,0
3,54,62,60,0,0,0,0,0,0,0
4,10,71,45,19,0,0,0,0,0,0
5,67,27,6,69,70,0,0,0,0,0
6,25,5,58,33,17,11,0,0,0,0
7,82,36,88,77,49,85,73,0,0,0
8,97,17,78,17,69,30,87,28,0,0
9,16,30,84,72,70,80,66,59,19,0


In [ ]:
!cat example.csv

City0,City1,City2,City3,City4,City5,City6,City7,City8,City9
0,0,0,0,0,0,0,0,0,0
51,0,0,0,0,0,0,0,0,0
22,8,0,0,0,0,0,0,0,0
54,62,60,0,0,0,0,0,0,0
10,71,45,19,0,0,0,0,0,0
67,27,6,69,70,0,0,0,0,0
25,5,58,33,17,11,0,0,0,0
82,36,88,77,49,85,73,0,0,0
97,17,78,17,69,30,87,28,0,0
16,30,84,72,70,80,66,59,19,0


In [ ]:
pd.read_csv('example.csv')

,City0,City1,City2,City3,City4,City5,City6,City7,City8,City9
0,0,0,0,0,0,0,0,0,0,0
1,51,0,0,0,0,0,0,0,0,0
2,22,8,0,0,0,0,0,0,0,0
3,54,62,60,0,0,0,0,0,0,0
4,10,71,45,19,0,0,0,0,0,0
5,67,27,6,69,70,0,0,0,0,0
6,25,5,58,33,17,11,0,0,0,0
7,82,36,88,77,49,85,73,0,0,0
8,97,17,78,17,69,30,87,28,0,0
9,16,30,84,72,70,80,66,59,19,0


In [ ]:
example.T

,0,1,2,3,4,5,6,7,8,9
City0,0,51,22,54,10,67,25,82,97,16
City1,0,0,8,62,71,27,5,36,17,30
City2,0,0,0,60,45,6,58,88,78,84
City3,0,0,0,0,19,69,33,77,17,72
City4,0,0,0,0,0,70,17,49,69,70
City5,0,0,0,0,0,0,11,85,30,80
City6,0,0,0,0,0,0,0,73,87,66
City7,0,0,0,0,0,0,0,0,28,59
City8,0,0,0,0,0,0,0,0,0,19
City9,0,0,0,0,0,0,0,0,0,0


In [ ]:
symmetric = pd.DataFrame(example.T.values + example.values, columns=example.columns)
symmetric

,City0,City1,City2,City3,City4,City5,City6,City7,City8,City9
0,0,51,22,54,10,67,25,82,97,16
1,51,0,8,62,71,27,5,36,17,30
2,22,8,0,60,45,6,58,88,78,84
3,54,62,60,0,19,69,33,77,17,72
4,10,71,45,19,0,70,17,49,69,70
5,67,27,6,69,70,0,11,85,30,80
6,25,5,58,33,17,11,0,73,87,66
7,82,36,88,77,49,85,73,0,28,59
8,97,17,78,17,69,30,87,28,0,19
9,16,30,84,72,70,80,66,59,19,0


In [ ]:
{ x : { y: symmetric[x][y] for y in cities if y != x } for x in cities} 

{'City0': {'City1': 51,
  'City2': 22,
  'City3': 54,
  'City4': 10,
  'City5': 67,
  'City6': 25,
  'City7': 82,
  'City8': 97,
  'City9': 16},
 'City1': {'City0': 51,
  'City2': 8,
  'City3': 62,
  'City4': 71,
  'City5': 27,
  'City6': 5,
  'City7': 36,
  'City8': 17,
  'City9': 30},
 'City2': {'City0': 22,
  'City1': 8,
  'City3': 60,
  'City4': 45,
  'City5': 6,
  'City6': 58,
  'City7': 88,
  'City8': 78,
  'City9': 84},
 'City3': {'City0': 54,
  'City1': 62,
  'City2': 60,
  'City4': 19,
  'City5': 69,
  'City6': 33,
  'City7': 77,
  'City8': 17,
  'City9': 72},
 'City4': {'City0': 10,
  'City1': 71,
  'City2': 45,
  'City3': 19,
  'City5': 70,
  'City6': 17,
  'City7': 49,
  'City8': 69,
  'City9': 70},
 'City5': {'City0': 67,
  'City1': 27,
  'City2': 6,
  'City3': 69,
  'City4': 70,
  'City6': 11,
  'City7': 85,
  'City8': 30,
  'City9': 80},
 'City6': {'City0': 25,
  'City1': 5,
  'City2': 58,
  'City3': 33,
  'City4': 17,
  'City5': 11,
  'City7': 73,
  'City8': 87,
  'City